In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import cPickle as pickle
import codecs
import skfuzzy as fuzz 
import time

from matplotlib import pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.cluster.bicluster import SpectralCoclustering
from biclustering.biclustering import DeltaBiclustering
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

In [2]:
%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})

In [3]:
arena_news_df = pd.read_pickle('arena_news_df.pkl')
sport_news_df = pd.read_pickle('sport_news_df.pkl')
jovem_news_df = pd.read_pickle('jovem_news_df.pkl')

In [4]:
labels_true = np.array(len(arena_news_df)*[0] + len(sport_news_df)*[1] + len(jovem_news_df.ix[0:99])*[2])

In [5]:
count_vect = CountVectorizer(encoding='UTF-8',lowercase=False, min_df=2)
X = count_vect.fit_transform(arena_news_df['all'].tolist() + sport_news_df['all'].tolist() + jovem_news_df['all'].ix[0:99].tolist())

X_train_norm_tfidf = TfidfTransformer(norm=u'l2', use_idf=True).fit_transform(X).toarray()
X_train_tfidf = TfidfTransformer(norm=False, use_idf=True).fit_transform(X).toarray()
X_train_norm = TfidfTransformer(norm=u'l2', use_idf=False).fit_transform(X).toarray()
X_train = TfidfTransformer(norm=False, use_idf=False).fit_transform(X).toarray()

In [6]:
print X_train.shape

(300, 6764)


In [7]:
def _big_s(x, center):
    len_x = len(x)
    total = 0

    for i in range(len_x):
        total += np.linalg.norm(x[i]-center)

    return total / len_x

def davies_bouldin_score(X, labels_pred, k_centers):
    try:
        num_clusters, _ = k_centers.shape
        big_ss = np.zeros([num_clusters], dtype=np.float64)
        d_eucs = np.zeros([num_clusters, num_clusters], dtype=np.float64)
        db = 0

        for k in range(num_clusters):
            samples_in_k_inds = np.where(labels_pred == k)[0]
            samples_in_k = X[samples_in_k_inds, :]
            big_ss[k] = _big_s(samples_in_k, k_centers[k])

        for k in range(num_clusters):
            for l in range(0, num_clusters):
                d_eucs[k, l] = np.linalg.norm(k_centers[k]-k_centers[l])

        for k in range(num_clusters):
            values = np.zeros([num_clusters-1], dtype=np.float64)
            for l in range(0, k):
                values[l] = (big_ss[k] + big_ss[l])/d_eucs[k, l]
            for l in range(k+1, num_clusters):
                values[l-1] = (big_ss[k] + big_ss[l])/d_eucs[k, l]

            db += np.max(values)
        res = db / num_clusters
    except Exception:
        return 0.0
    return res

def calculate_centroids_doc_mean(X, labels_pred, k):
    _, m = X.shape

    centroids = np.zeros((k, m))
    for k in range(k):
        samples_in_k_inds = np.where(labels_pred == k)[0]
        centroids[k, :] = X[samples_in_k_inds, :].mean(axis=0)

    return centroids

## K-means

In [9]:
params = {
    'k' : [3],
    'X' : ['X_train', 'X_train_norm', 'X_train_tfidf', 'X_train_norm_tfidf']
}
with codecs.open('kmeans_news_results.csv', 'w', 'utf-8') as out_f:
    out_f.write('X,K,NMI,RAND,DAVIES\n')
    for k in params['k']:
        for data_str in params['X']:
            data = eval(data_str)

            error_best = np.inf
            for _ in xrange(10):
                estimator = KMeans(n_clusters=k, max_iter=1000, init='random')
                estimator.fit(data)

                labels_pred = estimator.labels_
                centroids = estimator.cluster_centers_
                error = estimator.inertia_

                nmi_score = normalized_mutual_info_score(labels_true, labels_pred)
                rand_score = adjusted_rand_score(labels_true, labels_pred)
                davies_score = davies_bouldin_score(data, labels_pred, centroids)
                
                out_f.write(u'%s,%s,%s,%s,%s\n' % (data_str, k, nmi_score, rand_score, davies_score))

                print 'Execution: X: %s, k: %s' % (data_str, k)
                print 'NMI score: %s' % nmi_score
                print 'Rand score: %s' % rand_score
                print 'Davies score: %s' % davies_score
                print '-----------------------------------------------\n'

Execution: X: X_train, k: 3
NMI score: 0.115098348432
Rand score: 0.0176825500343
Davies score: 2.61251748031
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.0332641069195
Rand score: 4.48948747226e-05
Davies score: 0.460960431793
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.108523319497
Rand score: 0.00670394416546
Davies score: 1.90916798943
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.0798583744984
Rand score: 0.00718651227011
Davies score: 2.49414972016
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.0957653737681
Rand score: 0.00533421984794
Davies score: 2.21933882234
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.109527783943
Rand score: 0.0221321805431
Davies score: 2.53806177551
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score

## Fuzzy K-means

In [14]:
params = {
    'k' : [3],
    'X' : ['X_train', 'X_train_norm', 'X_train_tfidf', 'X_train_norm_tfidf']
}
with codecs.open('fuzzy_cmeans_news_results.csv', 'w', 'utf-8') as out_f:
    out_f.write('X,K,NMI,RAND,DAVIES\n')
    for k in params['k']:
        for data_str in params['X']:
            data = eval(data_str)

            error_best = np.inf
            for _ in xrange(10):
                centroids, U, _, _, errors, _, _ = fuzz.cluster.cmeans(
                    data.T,
                    k,
                    2,
                    error=0.00001,
                    maxiter=10000)
                centroids

                labels_pred = np.argmax(U, axis=0)
                error = errors[-1]

                nmi_score = normalized_mutual_info_score(labels_true, labels_pred)
                rand_score = adjusted_rand_score(labels_true, labels_pred)
                davies_score = davies_bouldin_score(data, labels_pred, centroids)
                
                out_f.write(u'%s,%s,%s,%s,%s\n' % (data_str, k, nmi_score, rand_score, davies_score))

                print 'Execution: X: %s, k: %s' % (data_str, k)
                print 'NMI score: %s' % nmi_score
                print 'Rand score: %s' % rand_score
                print 'Davies score: %s' % davies_score
                print '-----------------------------------------------\n'

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
-----------------------------------------------

Execution: X: X_train, k: 3
NMI score: 0.069433856787
Rand score: 0.0671231171273
Davies score: 0.0
------

## Orthogonal Non-negative Matrix Tri-Factorization

In [ ]:
def onmtf(X, U, S, V):
    U = U * ((X.dot(V).dot(S.T)) / (U.dot(S).dot(V.T).dot(X.T).dot(U)))
    V = V * ((X.T.dot(U).dot(S)) / (V.dot(S.T).dot(U.T).dot(X).dot(V)))
    S = S * ((U.T.dot(X).dot(V)) / (U.T.dot(U).dot(S).dot(V.T).dot(V)))
    return U, S, V

def onm3f(X, U, S, V):
     U = U * (X.dot(V).dot(S.T)) / np.sqrt(U.dot(U.T).dot(X).dot(V).dot(S.T))
     V = V * (X.T.dot(U).dot(S)) / np.sqrt(V.dot(V.T).dot(X.T).dot(U).dot(S))
     S = S * (U.T.dot(X).dot(V)) / np.sqrt(U.T.dot(U).dot(S).dot(V.T).dot(V))
     return U, S, V

def nbvd(X, U, S, V):
     U = U * (X.dot(V).dot(S.T)) / U.dot(U.T).dot(X).dot(V).dot(S.T)
     V = V * (X.T.dot(U).dot(S)) / V.dot(V.T).dot(X.T).dot(U).dot(S)
     S = S * (U.T.dot(X).dot(V)) / U.T.dot(U).dot(S).dot(V.T).dot(V)
     return U, S, V
    
def matrix_factorization_clustering(X, k, l, factorization_func=onmtf, norm=False, num_iters=100):
    m, n = X.shape
    U = np.random.rand(m,k)
    S = np.random.rand(k,l)
    V = np.random.rand(n,l)

    if norm:
        X = Normalizer().fit_transform(X)

    error_best = np.inf

    for i in xrange(num_iters):
        U, S, V = factorization_func(X, U, S, V)
        error = np.sum((X - U.dot(S).dot(V.T)) ** 2)
        
        if error < error_best:
            U_best = U
            S_best = S
            V_best = V
            error_best = error

    Du = np.diag(np.ones(m).dot(U_best))
    Dv = np.diag(np.ones(n).dot(V_best))

    U_norm = U_best.dot( np.diag(S_best.dot(Dv).dot(np.ones(l))) )
    V_norm = V_best.dot( np.diag(np.ones(k).dot(Du).dot(S_best)) )

    rows_ind = np.argmax(U_best, axis=1)
    cols_ind = np.argmax(V_best, axis=1)

    return U_norm, S_best, V_norm, rows_ind, cols_ind, error_best


params = {
    'k' : [3],
    'l' : [2, 3, 4, 5, 6],
#     'X' : ['X_train', 'X_train_tfidf']
    'X' : ['X_train', 'X_train_norm', 'X_train_tfidf', 'X_train_norm_tfidf']
#     'X' : ['X_train_norm', 'X_train_norm_tfidf']
}
with codecs.open('onmtf_news_results.csv', 'w', 'utf-8') as out_f:
    out_f.write('X,K,L,NMI,RAND,DAVIES\n')
    for k in params['k']:
        for l in params['l']:
            for data_str in params['X']:
                data = eval(data_str)

                error_best = np.inf
                for _ in xrange(10):
                    init_time = time.time()
                    U, S, V, labels_pred, _, error = matrix_factorization_clustering(data, k, l, num_iters=1000)

                    nmi_score = normalized_mutual_info_score(labels_true, labels_pred)
                    rand_score = adjusted_rand_score(labels_true, labels_pred)
                    davies_score = davies_bouldin_score(data, labels_pred, calculate_centroids_doc_mean(data, labels_pred, k))

                    end_time = time.time()
                    print end_time - init_time

                    out_f.write(u'%s,%s,%s,%s,%s,%s\n' % (data_str, k, l, nmi_score, rand_score, davies_score))

                    print 'Execution: X: %s, k: %s' % (data_str, k)
                    print 'Algo error: %s' % error
                    print 'NMI score: %s' % nmi_score
                    print 'Rand score: %s' % rand_score
                    print 'Davies score: %s' % davies_score
                    print '-----------------------------------------------\n'

880.380249023
Execution: X: X_train, k: 3
Algo error: inf
NMI score: 0.105763880717
Rand score: 0.112250589335
Davies score: 6.60967741576
-----------------------------------------------

961.614548206
Execution: X: X_train, k: 3
Algo error: inf
NMI score: 0.00955425382046
Rand score: -0.000504490234317
Davies score: 3.60430111693
-----------------------------------------------

988.1425879
Execution: X: X_train, k: 3
Algo error: inf
NMI score: 0.112166021861
Rand score: 0.0930987431886
Davies score: 7.26283227383
-----------------------------------------------

2016.82614207
Execution: X: X_train, k: 3
Algo error: inf
NMI score: 0.0114363896112
Rand score: 0.000152143507417
Davies score: 3.95778514682
-----------------------------------------------

956.24012804
Execution: X: X_train, k: 3
Algo error: inf
NMI score: 0.074377378081
Rand score: 0.0806982433705
Davies score: 7.25519125519
-----------------------------------------------

1102.90737486
Execution: X: X_train, k: 3
Algo erro

## Fast Non-negative Matrix Tri-Factorization

In [ ]:
def fnmtf(X, k, l, num_iter=1000, norm=False):
    m, n = X.shape

    def weights_initialization(X, n, m, k):
        shuffle_inds = np.random.permutation(n)
        cluster_end_ind = 0
        for i in xrange(k):
            cluster_init_ind = cluster_end_ind
            cluster_end_ind = round((i + 1) * n / k)
            X[shuffle_inds[cluster_init_ind : cluster_end_ind], i] = 1
        return X
    
    U = weights_initialization(np.zeros((m, k)), m, n, k)
    S = np.random.rand(k,l)
    V = weights_initialization(np.zeros((n, l)), n, m, l)

    error_best = np.inf
    error = error_best

    if norm:
        X = Normalizer().fit_transform(X)
    for _ in xrange(num_iter):
        S = np.linalg.pinv(U.T.dot(U)).dot(U.T).dot(X).dot(V).dot(np.linalg.pinv(V.T.dot(V)))

        # solve subproblem to update V
        U_tilde = U.dot(S)
        V_new = np.zeros(n*l).reshape(n, l)
        for j in xrange(n):
            errors = np.zeros(l)
            for col_clust_ind in xrange(l):
                errors[col_clust_ind] = ((X[:][:, j] - U_tilde[:][:, col_clust_ind])**2).sum()
            ind = np.argmin(errors)
            V_new[j][ind] = 1
        V = V_new

#         while np.linalg.det(V.T.dot(V)) <= 0:
#             if np.isnan( np.sum(V) ):
#                 break

#             erros = (X - U.dot(S).dot(V.T)) ** 2
#             erros = np.sum(erros.dot(V), axis=0) / np.sum(V, axis=0)
#             erros[np.where(np.sum(V, axis=0) <= 1)] = -np.inf
#             quantidade = np.sum(V, axis=0)
#             indexMin = np.argmin(quantidade)
#             indexMax = np.argmax(erros)
#             indexes = np.nonzero(V[:, indexMax])[0]
#             end = len(indexes)
#             indexes_p = np.random.permutation(end)
#             V[indexes[indexes_p[0:np.floor(end/2.0)]], indexMax] = 0.0
#             V[indexes[indexes_p[0:np.floor(end/2.0)]], indexMin] = 1.0

        # solve subproblem to update U
        V_tilde = S.dot(V.T)
        U_new = np.zeros(m*k).reshape(m, k)
        for i in xrange(m):
            errors = np.zeros(k)
            for row_clust_ind in xrange(k):
                errors[row_clust_ind] = ((X[i][:] - V_tilde[row_clust_ind][:])**2).sum()
            ind = np.argmin(errors)
            U_new[i][ind] = 1
        U = U_new

#         while np.linalg.det(U.T.dot(U)) <= 0:
#             if np.isnan( np.sum(U) ):
#                 break

#             erros = (X - U.dot(V_tilde)) ** 2
#             erros = np.sum(U.T.dot(erros), axis=1) / np.sum(U, axis=0)
#             erros[np.where(np.sum(U, axis=0) <= 1)] = -np.inf
#             quantidade = np.sum(U, axis=0)
#             indexMin = np.argmin(quantidade)
#             indexMax = np.argmax(erros)
#             indexes = np.nonzero(U[:, indexMax])[0]

#             end = len(indexes)
#             indexes_p = np.random.permutation(end)
#             U[indexes[indexes_p[0:np.floor(end/2.0)]], indexMax] = 0.0
#             U[indexes[indexes_p[0:np.floor(end/2.0)]], indexMin] = 1.0

        error_ant = error
#         print error_ant
        error = np.sum((X - U.dot(S).dot(V.T)) ** 2)

        if error < error_best:
            U_best = U
            S_best = S
            V_best = V
            error_best = error

#         if np.abs(error - error_ant) <= 0.000001:
#             break

    rows_ind = np.argmax(U, axis=1)
    cols_ind = np.argmax(V, axis=1)

    return U, S, V, rows_ind, cols_ind, error


params = {
    'k' : [3],
    'l' : [2, 3, 4, 5, 6],
    'X' : ['X_train', 'X_train_norm', 'X_train_tfidf', 'X_train_norm_tfidf']
#     'X' : ['X_train', 'X_train_tfidf']
}

with codecs.open('nmtf_bin_news_results.csv', 'w', 'utf-8') as out_f:
    out_f.write('X,K,L,NMI,RAND,DAVIES\n')
    for k in params['k']:
        for l in params['l']:
            for data_str in params['X']:
                data = eval(data_str)

                error_best = np.inf
                for _ in xrange(10):
                    init_time = time.time()
                    U, S, V, labels_pred, _, error = fnmtf(data, k, l)

                    nmi_score = normalized_mutual_info_score(labels_true, labels_pred)
                    rand_score = adjusted_rand_score(labels_true, labels_pred)
                    davies_score = davies_bouldin_score(data, labels_pred, calculate_centroids_doc_mean(data, labels_pred, k))

#                     if error < error_best:
#                         error_best = error
#                         nmi_score_best = nmi_score
#                         rand_score_best = rand_score
#                         davies_score_best = davies_score
#                         labels_pred_best = labels_pred

                    end_time = time.time()
                    print end_time - init_time

                    out_f.write(u'%s,%s,%s,%s,%s,%s\n' % (data_str, k, l, nmi_score, rand_score, davies_score))

                    print 'Execution: X: %s, k: %s, l: %s' % (data_str, k, l)
                    print 'Algo error: %s' % error
                    print 'NMI score: %s' % nmi_score
                    print 'Rand score: %s' % rand_score
                    print 'Davies score: %s' % davies_score
                    print '-----------------------------------------------\n'

171.905370951
Execution: X: X_train, k: 3, l: 2
Algo error: 212713.143414
NMI score: 0.0558855682487
Rand score: 0.0352532594765
Davies score: 5.04271402634
-----------------------------------------------

170.441447973
Execution: X: X_train, k: 3, l: 2
Algo error: 212713.4328
NMI score: 0.0587342988573
Rand score: 0.0382849986737
Davies score: 5.04166124195
-----------------------------------------------

170.962042093
Execution: X: X_train, k: 3, l: 2
Algo error: 212713.143414
NMI score: 0.0558855682487
Rand score: 0.0352532594765
Davies score: 5.04271402634
-----------------------------------------------

170.195071936
Execution: X: X_train, k: 3, l: 2
Algo error: 212713.143414
NMI score: 0.0558855682487
Rand score: 0.0352532594765
Davies score: 5.04271402634
-----------------------------------------------

170.566911936
Execution: X: X_train, k: 3, l: 2
Algo error: 212713.4328
NMI score: 0.0587342988573
Rand score: 0.0382849986737
Davies score: 5.04166124195
-----------------------

## Fast Overlapping Non-negative Matrix Tri-Factorization

In [ ]:
def matrix_factorization_overlapping_bin(X, k, l, num_iters=1000):
    def weights_initialization(X, n, m, k):
        shuffle_inds = np.random.permutation(n)
        cluster_end_ind = 0
        for i in xrange(k):
            cluster_init_ind = cluster_end_ind
            cluster_end_ind = round((i + 1) * n / k)
            X[shuffle_inds[cluster_init_ind : cluster_end_ind], i] = 1
        return X

    def calculate_block_matrix(X, F, G, S, k, l):
        for i in xrange(k):
            for j in xrange(l):
                S[i, j] = np.mean(X[F[:, i] == 1][:, G[i][:, j] == 1])
        where_are_NaNs = np.isnan(S)
        S[where_are_NaNs] = 0
        return S

    n, m = X.shape

    error_best = np.inf
    error = np.inf

    F = weights_initialization(np.zeros((n, k)), n, m, k)

    G = []
    for i in xrange(k):
        G.append( weights_initialization(np.zeros((m, l)), m, n, l) )

    S = np.random.rand(k, l)

    for iter_ind in xrange(num_iters):
        S = calculate_block_matrix(X, F, G, S, k, l)

        # Update G
        for i in xrange(k):
            F_t = F[F[:, i] == 1, :].dot(S)
            X_t = X[F[:, i] == 1, :]
            G[i] = np.zeros((m, l))
            for j in xrange(m):
                clust_len, _ = X_t.shape
                diff = F_t - X_t[:, j].reshape(clust_len, 1).dot(np.ones(l).reshape(1, l))
                errors = np.diag(diff.T.dot(diff))
                minV = np.min(errors)
                index = np.where(errors <= minV)[0]
                G[i][j, index[np.random.randint(len(index))]] = 1

#             while np.linalg.det(G[i].T.dot(G[i])) <= 0:
#                 erros = (X_t - F_t.dot(G[i].T)) ** 2
#                 erros = np.sum(erros.dot(G[i]), axis=0) / np.sum(G[i], axis=0)
#                 erros[np.where(np.sum(G[i], axis=0) <= 1)] = -np.inf
#                 quantidade = np.sum(G[i], axis=0)
#                 indexMin = np.argmin(quantidade)
#                 indexMax = np.argmax(erros)
#                 indexes = np.nonzero(G[i][:, indexMax])[0]

#                 end = len(indexes)
#                 indexes_p = np.random.permutation(end)
#                 G[i][indexes[indexes_p[0:np.floor(end/2.0)]], indexMax] = 0.0
#                 G[i][indexes[indexes_p[0:np.floor(end/2.0)]], indexMin] = 1.0

#         S = calculate_block_matrix(X, F, G, S, k, l)

        G_t = np.zeros((k, m))
        for i in xrange(k):
            G_t[i, :] = S[i, :].dot(G[i].T)

        F = np.zeros((n, k))
        for j in xrange(n):
            diff = G_t - np.ones(k).reshape(k, 1).dot(X[j, :].reshape(1, m))
            errors = np.diag(diff.dot(diff.T))
            minV = np.min(errors)
            index = np.where(errors <= minV)[0]
            F[j, index[np.random.randint(len(index))]] = 1

#         while np.linalg.det(F.T.dot(F)) <= 0:
#             erros = (X - F.dot(G_t)) ** 2
#             erros = np.sum(F.T.dot(erros), axis=1) / np.sum(F, axis=0)
#             erros[np.where(np.sum(F, axis=0) <= 1)] = -np.inf
#             quantidade = np.sum(F, axis=0)
#             indexMin = np.argmin(quantidade)
#             indexMax = np.argmax(erros)
#             indexes = np.nonzero(F[:, indexMax])[0]

#             end = len(indexes)
#             indexes_p = np.random.permutation(end)
#             F[indexes[indexes_p[0:np.floor(end/2.0)]], indexMax] = 0.0
#             F[indexes[indexes_p[0:np.floor(end/2.0)]], indexMin] = 1.0

        error_ant = error
        error = np.sum((X - F.dot(G_t))**2)
#         print error

        if error < error_best:
            error_best = error
            F_best = F
            S_best = S
            G_best = G
            G_t_best = G_t

#         if np.abs(error - error_ant) <= 0.000001:
#             break

    rows_ind = np.argmax(F_best, axis=1)
    reconstruction = F_best.dot(G_t_best)

    return F, S, G, G_t, rows_ind, error_best, reconstruction


params = {
    'k' : [3],
    'l' : [2, 3, 4, 5, 6],
    'X' : ['X_train', 'X_train_norm', 'X_train_tfidf', 'X_train_norm_tfidf']
}

with codecs.open('ovnmtf_bin_news_results.csv', 'w', 'utf-8') as out_f:
    out_f.write('X,K,L,NMI,RAND,DAVIES\n')
    for k in params['k']:
        for l in params['l']:
            for data_str in params['X']:
                data = eval(data_str)

                error_best = np.inf
                for _ in xrange(10):
                    init_time = time.time()
                    U, S, V, V_t, labels_pred, error, _ = matrix_factorization_overlapping_bin(data, k, l)

                    nmi_score = normalized_mutual_info_score(labels_true, labels_pred)
                    rand_score = adjusted_rand_score(labels_true, labels_pred)
                    davies_score = davies_bouldin_score(data, labels_pred, calculate_centroids_doc_mean(data, labels_pred, k))

#                     if error < error_best:
#                         error_best = error
#                         nmi_score_best = nmi_score
#                         rand_score_best = rand_score
#                         davies_score_best = davies_score
#                         labels_pred_best = labels_pred

                    end_time = time.time()
                    print end_time - init_time

                    out_f.write(u'%s,%s,%s,%s,%s,%s\n' % (data_str, k, l, nmi_score, rand_score, davies_score))

                    print 'Execution: X: %s, k: %s, l: %s' % (data_str, k, l)
                    print 'Algo error: %s' % error
                    print 'NMI score: %s' % nmi_score
                    print 'Rand score: %s' % rand_score
                    print 'Davies score: %s' % davies_score
                print '-----------------------------------------------\n'

## Overlapping Non-Negative Matrix Tri-Factorization

In [ ]:
def is_any_clust_empty(U_bin):
    n, k = U_bin.shape
    return np.count_nonzero(np.sum(U_bin, axis=0)) != k

def overlapping_matrix_factorization_coclustering(X, k, l, norm=False, num_iters=100):
    n, m = X.shape
    U = np.random.rand(n, k)
    S = np.random.rand(k, l)
    V = []
    for i in xrange(k):
        V.append(np.random.rand(m, l))

    Ii = np.zeros((k, k))
    Ij = np.zeros((k, k))

    error_best = np.inf
    
    if norm:
        X = Normalizer().fit_transform(X)

    V_tilde = np.zeros((k, m))
    for i in xrange(k):
        Ii[i, i] = 1
        V_tilde += Ii.dot(S).dot(V[i].T)
        Ii[i, i] = 0
    error = np.sum((X - U.dot(V_tilde)) ** 2)

    for _ in xrange(num_iters):
        # Update U
        new_U_pos = np.zeros((n, k))
        new_U_neg = np.zeros((n, k))
        for i in xrange(k):
            Ii[i, i] = 1
            for j in xrange(k):
                Ij[j, j] = 1
                new_U_pos += U.dot(Ii).dot(S).dot(V[i].T).dot(V[j]).dot(S.T).dot(Ij)
                Ij[j, j] = 0
            new_U_neg += X.dot(V[i]).dot(S.T).dot(Ii)
            Ii[i, i] = 0
        U = U * (new_U_neg / new_U_pos)

        # Compute V'
        V_tilde = np.zeros((k, m))
        for i in xrange(k):
            Ii[i, i] = 1
            V_tilde += Ii.dot(S).dot(V[i].T)
            Ii[i, i] = 0

        # Update Vi
        for i in xrange(k):
            new_V_pos = np.zeros((m, l))
            new_V_neg = np.zeros((m, l))
            Ii[i, i] = 1
            for j in xrange(k):
                Ij[j, j] = 1

                new_V_pos += V[j].dot(S.T).dot(Ij).dot(U.T).dot(U).dot(Ii).dot(S)

                Ij[j, j] = 0

            new_V_neg += X.T.dot(U).dot(Ii).dot(S)

            Ii[i, i] = 0
            V[i] = V[i] * (new_V_neg / new_V_pos)

        # Recompute V'
        V_tilde = np.zeros((k, m))
        for i in xrange(k):
            Ii[i, i] = 1
            V_tilde += Ii.dot(S).dot(V[i].T)
            Ii[i, i] = 0
            
        new_S_pos = np.zeros((k, l))
        new_S_neg = np.zeros((k, l))
        for i in xrange(k):
            Ii[i, i] = 1
            for j in xrange(k):
                Ij[j, j] = 1
                new_S_pos += Ii.dot(U.T).dot(U).dot(Ij).dot(S).dot(V[j].T).dot(V[i])
                Ij[j, j] = 0
            new_S_neg += Ii.dot(U.T).dot(X).dot(V[i])
            Ii[i, i] = 0
        S = S * (new_S_neg / new_S_pos)

#         import pdb; pdb.set_trace()

        V_tilde = np.zeros((k, m))
        for i in xrange(k):
            Ii[i, i] = 1
            V_tilde += Ii.dot(S).dot(V[i].T)
            Ii[i, i] = 0

        error_ant = error
        error = np.sum((X - U.dot(V_tilde))**2)
#         print errorV_t

        if error < error_best:
            error_best = error
            U_best = U
            S_best = S
            V_best = V

        if np.abs(error - error_ant) <= 0.000001:
            break


    rows_ind = np.argmax(U_best, axis=1)
    reconstruction = U_best.dot(V_tilde)

    return U_best, S_best, V_best, V_tilde, rows_ind, error_best, reconstruction


params = {
    'k' : [3],
    'l' : [2, 3, 4, 5, 6],
    'X' : ['X_train', 'X_train_norm', 'X_train_tfidf', 'X_train_norm_tfidf']
}

with codecs.open('ovnmtf_news_results.csv', 'w', 'utf-8') as out_f:
    out_f.write('X,K,L,NMI,RAND,DAVIES\n')
    for k in params['k']:
        for l in params['l']:
            for data_str in params['X']:
                data = eval(data_str)

                error_best = np.inf
                for _ in xrange(10):
                    init_time = time.time()
                    U, S, V, V_t, labels_pred, error, _ = overlapping_matrix_factorization_coclustering(data, k, l)

                    nmi_score = normalized_mutual_info_score(labels_true, labels_pred)
                    rand_score = adjusted_rand_score(labels_true, labels_pred)
                    davies_score = davies_bouldin_score(data, labels_pred, calculate_centroids_doc_mean(data, labels_pred, k))

#                     if error < error_best:
#                         error_best = error
#                         nmi_score_best = nmi_score
#                         rand_score_best = rand_score
#                         davies_score_best = davies_score
#                         labels_pred_best = labels_pred

                    end_time = time.time()
                    print end_time - init_time

                out_f.write(u'%s,%s,%s,%s,%s,%s\n' % (data_str, k, l, nmi_score, rand_score, davies_score))

                print 'Execution: X: %s, k: %s, l: %s' % (data_str, k, l)
                print 'Algo error: %s' % error
                print 'NMI score: %s' % nmi_score
                print 'Rand score: %s' % rand_score
                print 'Davies score: %s' % davies_score
                print '-----------------------------------------------\n'